
# Bayesian Learning με Naive Bayes σε SMS Spam dataset

Σε αυτό το notebook υλοποιούμε ένα απλό παράδειγμα **Bayesian learning**
για ταξινόμηση κειμένου με χρήση του **Multinomial Naive Bayes**.

## Στόχοι

- Να συνδέσουμε τον τύπο του Bayes με ένα πραγματικό πρόβλημα ταξινόμησης.
- Να δούμε πώς η υπόθεση ανεξαρτησίας των χαρακτηριστικών οδηγεί στο Naive Bayes.
- Να εκπαιδεύσουμε έναν ταξινομητή για SMS *ham* / *spam* με scikit-learn.
- Να παρατηρήσουμε εποπτικά την απόδοση του μοντέλου (confusion matrix, κ.λπ.).


## Θεωρία: Bayes & Naive Bayes

Θυμόμαστε τον τύπο του Bayes:

$$
P(y \mid x) = \frac{P(x \mid y) P(y)}{P(x)}
$$

όπου:

- $y$ είναι η κλάση (π.χ. $y \in \{\text{ham}, \text{spam}\}$),
- $x$ είναι το διάνυσμα χαρακτηριστικών (π.χ. οι λέξεις του SMS),
- $P(y)$ είναι η **prior** πιθανότητα της κλάσης,
- $P(x \mid y)$ είναι η **likelihood**,
- $P(y \mid x)$ είναι η **posterior** πιθανότητα.

Στο **Naive Bayes** υποθέτουμε ότι τα χαρακτηριστικά $x_i$ είναι
conditionally independent δεδομένης της κλάσης:

$$
P(x \mid y) = \prod_i P(x_i \mid y)
$$

Έτσι, η posterior γράφεται (παραλείποντας το σταθερό $P(x)$):

$$
P(y \mid x) \propto P(y) \prod_i P(x_i \mid y)
$$

Στο σενάριο ταξινόμησης κειμένου, οι $x_i$ αντιστοιχούν σε λέξεις
ή n-grams, και το μοντέλο **Multinomial Naive Bayes** χρησιμοποιεί
τις συχνότητες εμφάνισης των λέξεων σε κάθε κλάση.

> **Σημείωση**: Για λεπτομερέστερη ανάλυση της μετατροπής κειμένου σε αριθμητικά δεδομένα 
> (Bag-of-Words, TF-IDF, stopwords κλπ.), δες το 
> [**Bayesian Learning README**](../bayesian_learning/README.md#211-θεωρία-nlp-bag-of-words-λεξιλόγιο--tf-idf) 
> όπου εξηγούνται αναλυτικά όλες οι τεχνικές NLP που χρησιμοποιούμε.


In [ ]:
# Βασικά imports για ανάλυση δεδομένων και machine learning

from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
    accuracy_score,
    precision_recall_fscore_support,
)

plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["font.size"] = 11


ROOT = Path('..')
DATA_PATH = ROOT / 'data' / 'sms_spam.csv'

print(f"Θα φορτώσουμε τα δεδομένα από: {DATA_PATH}")

In [ ]:
# Φόρτωση του dataset sms_spam.csv από τον φάκελο data/
# Χρησιμοποιούμε encoding='latin-1' γιατί το αρχείο δεν είναι UTF-8

df = pd.read_csv(DATA_PATH, encoding='latin-1')
print(f"Αρχικό σχήμα dataset: {df.shape}")

In [ ]:
# Ελέγχουμε τι στήλες υπάρχουν στο DataFrame

print("Στήλες του DataFrame:")
print(df.columns.tolist())
print()
print("Πρώτες γραμμές:")
print(df.head())


In [ ]:

# Βασικός καθαρισμός: αφαιρούμε τις περιττές στήλες και εγγραφές με κενά
# Κρατάμε μόνο τις στήλες v1 (label) και v2 (text)
df = df[['v1', 'v2']].copy()
df.columns = ['label', 'text']  # Μετονομάζουμε για ευκολία
df = df.dropna(subset=['label', 'text']).copy()

# Κανονικοποίηση labels
df['label'] = df['label'].str.strip().str.lower()
df = df[df['label'].isin(['ham', 'spam'])].copy()

print(f"Σχήμα μετά τον καθαρισμό (μόνο ham/spam): {df.shape}")
df.head()


In [ ]:
# Αφαίρεση stopwords (συχνές λέξεις που δεν προσθέτουν πληροφορία)
# Π.χ. "the", "a", "is", "and" κλπ.

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

print("Παράδειγμα stopwords:", list(ENGLISH_STOP_WORDS)[:20])
print(f"Σύνολο stopwords: {len(ENGLISH_STOP_WORDS)}")


In [ ]:
# Ελέγχουμε την κατανομή των κλάσεων ham / spam

class_counts = df["label"].value_counts().sort_index()
print("Κατανομή κλάσεων:")
print(class_counts)

fig, ax = plt.subplots()
ax.bar(class_counts.index, class_counts.values)
ax.set_title("Κατανομή κλάσεων στο SMS Spam dataset")
ax.set_xlabel("Κλάση")
ax.set_ylabel("Πλήθος μηνυμάτων")
plt.tight_layout()
plt.show()

## Αξιολόγηση Μοντέλου: Μετρικές & Confusion Matrix

Όταν εκπαιδεύουμε ένα ταξινομητή (π.χ. για ham / spam), δεν μας αρκεί να ξέρουμε ότι
«δουλεύει καλά». Θέλουμε να μετρήσουμε **πόσο καλά** τα πάει και **τι είδους λάθη** κάνει.

Σε αυτή την ενότητα θα δούμε:

1. Την **confusion matrix** και τη βασική ορολογία (TP, FP, TN, FN).
2. Πώς από αυτά τα μεγέθη προκύπτουν οι μετρικές **Precision**, **Recall**, **F1-score** και **Support**.

---

### 1. Confusion Matrix και βασικές έννοιες

Ας υποθέσουμε ότι έχουμε ένα πρόβλημα δύο κλάσεων:

- **spam** = θετική κλάση (positive)
- **ham**  = αρνητική κλάση (negative)

Το μοντέλο μας, για κάθε μήνυμα, κάνει μια πρόβλεψη (spam ή ham), που μπορεί να είναι σωστή ή λάθος.
Συγκρίνοντας τις **προβλέψεις** με τις **πραγματικές ετικέτες**, μπορούμε να μετρήσουμε:

- **TP (True Positive)**: Μηνύματα που ήταν spam και το μοντέλο τα πρόβλεψε ως spam.
- **TN (True Negative)**: Μηνύματα που ήταν ham και το μοντέλο τα πρόβλεψε ως ham.
- **FP (False Positive)**: Μηνύματα που ήταν ham αλλά το μοντέλο τα πρόβλεψε ως spam (*false alarm*).
- **FN (False Negative)**: Μηνύματα που ήταν spam αλλά το μοντέλο τα πρόβλεψε ως ham (*χαμένα spam*).

Αυτά τα τέσσερα μεγέθη οργανώνονται στον **confusion matrix**:

$$
\begin{pmatrix}
\text{TN} & \text{FP} \\
\text{FN} & \text{TP}
\end{pmatrix}
$$

- Τα διαγώνια στοιχεία (TN, TP) είναι οι **σωστές προβλέψεις**.
- Τα μη διαγώνια στοιχεία (FP, FN) είναι τα **λάθη** του μοντέλου.

---

### 2. Μετρικές ταξινόμησης

Χρησιμοποιώντας τα TP, FP, TN, FN μπορούμε να ορίσουμε διάφορες μετρικές.
Για το spam filtering, μας ενδιαφέρει κυρίως η απόδοση ως προς την κλάση **spam** (θετική κλάση).

#### 2.1 Precision (Ακρίβεια θετικών προβλέψεων)

Η **Precision** απαντάει στην ερώτηση:

> Από όλα τα μηνύματα που το μοντέλο χαρακτήρισε ως *spam*, πόσα ήταν πράγματι spam;

Με βάση τα TP και FP ορίζεται ως:

$$
\text{Precision} = \frac{\text{TP}}{\text{TP} + \text{FP}}
$$

- Υψηλή Precision σημαίνει ότι όταν το μοντέλο λέει «spam», **σπάνια κάνει λάθος**.
- Είναι σημαντική όταν θέλουμε να αποφύγουμε τα **false positives** (να μην μπλοκάρουμε κανονικά μηνύματα).
- Παράδειγμα: Αν Precision=0.95, σημαίνει ότι το 95% των μηνυμάτων που προβλέψαμε ως spam ήταν πραγματικά spam.

#### 2.2 Recall (Ανάκληση / Ευαισθησία)

Η **Recall** απαντάει στην ερώτηση:

> Από όλα τα μηνύματα που ήταν πράγματι *spam*, πόσα κατάφερε να βρει το μοντέλο;

Ορίζεται ως:

$$
\text{Recall} = \frac{\text{TP}}{\text{TP} + \text{FN}}
$$

- Υψηλή Recall σημαίνει ότι **δεν χάνουμε πολλά spam** (λίγα FN).
- Είναι σημαντική όταν μας ενδιαφέρει να εντοπίσουμε **όσο το δυνατόν περισσότερα** spam μηνύματα.
- Παράδειγμα: Αν Recall=0.92, σημαίνει ότι βρήκαμε το 92% από όλα τα αληθινά spam μηνύματα.


#### 2.3 F1-score

Συχνά θέλουμε **ένα ενιαίο νούμερο** που να συνδυάζει και την Precision και την Recall.
Ο **F1-score** είναι ο **αρμονικός μέσος** των Precision και Recall:

$$
\text{F1} = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}
$$

Ιδιότητες:

- Παίρνει τιμές από 0 έως 1.
- Είναι υψηλός μόνο όταν είναι υψηλές **και** η Precision **και** η Recall.
- Χρήσιμος όταν θέλουμε ένα **ισορροπημένο μέτρο** απόδοσης.

#### 2.4 Support

Το **support** δεν είναι «ποιότητα» του μοντέλου, αλλά απλά:

> το πλήθος των παραδειγμάτων κάθε κλάσης στο **validation set** (ή test set).

Στο `classification_report` της scikit-learn θα δεις για κάθε κλάση:

- Precision  
- Recall  
- F1-score  
- Support  → πόσα δείγματα αυτής της κλάσης υπήρχαν στο σύνολο αξιολόγησης.

Το support σε βοηθά να κρίνεις **πόσο αξιόπιστες** είναι οι μετρικές μιας κλάσης  
(π.χ. F1-score σε μια κλάση με μόνο 3 δείγματα δεν είναι πολύ σταθερό μέτρο).

---

### 3. Πώς τα χρησιμοποιούμε στην πράξη;

1. Φτιάχνουμε προβλέψεις του μοντέλου σε ένα **validation set** (ή test set).  
2. Υπολογίζουμε TP, FP, TN, FN → σχηματίζουμε τον **confusion matrix**.  
3. Από αυτά υπολογίζουμε:
   - Precision (για το spam),
   - Recall (για το spam),
   - F1-score,
   - και βλέπουμε το support κάθε κλάσης.
4. Συγκρίνουμε διαφορετικά μοντέλα ή διαφορετικές ρυθμίσεις (π.χ. τιμές του $\alpha$)
   με βάση αυτές τις μετρικές, για να διαλέξουμε την παραμετροποίηση με την καλύτερη συνολική συμπεριφορά.

---

### 4. Συνοπτικά μέτρα: accuracy, macro avg, weighted avg

Εκτός από τις μετρικές ανά κλάση (precision, recall, F1, support), το `classification_report` της scikit-learn
εμφανίζει και συνοπτικά μέτρα για όλο το μοντέλο: **accuracy**, **macro avg** και **weighted avg**.

#### Accuracy

Η **accuracy** (συνολική ακρίβεια) μετράει ποιο ποσοστό των δειγμάτων ταξινομήθηκε σωστά:

$$
\text{Accuracy} = \frac{\text{TP} + \text{TN}}{\text{TP} + \text{TN} + \text{FP} + \text{FN}}
$$

- Αριθμητής: όλα τα **σωστά** (True Positives + True Negatives).
- Παρονομαστής: **όλα** τα δείγματα (σωστά + λάθη).
- Δείχνει «πόσο συχνά έχει δίκιο» συνολικά το μοντέλο, χωρίς να ξεχωρίζει κλάσεις.

#### macro avg

Για κάθε μετρική (π.χ. precision ή recall ή F1) μπορούμε να υπολογίσουμε πρώτα την τιμή της **ανά κλάση**,
π.χ. $ m_1, m_2, \dots, m_K $ για $ K $ κλάσεις, και μετά τον **απλό μέσο όρο**:

$$
m_{\text{macro}} = \frac{1}{K} \sum_{k=1}^{K} m_k
$$

- Κάθε κλάση έχει **ίσο βάρος**, ανεξάρτητα από το πόσα δείγματα έχει.
- Χρήσιμο όταν μας ενδιαφέρει η **μέση απόδοση ανά κλάση**, π.χ. να μη «θυσιάζουμε» μια μικρή κλάση.

Στο `classification_report`, το `macro avg` δίνεται χωριστά για precision, recall και F1-score.

#### weighted avg

Ο **weighted avg** είναι κι αυτός μέσος όρος πάνω στις κλάσεις, αλλά τώρα κάθε κλάση
ζυγίζεται με βάση το **support** της (πόσα δείγματα έχει).

Αν \( \text{support}_k \) είναι το πλήθος δειγμάτων της κλάσης \( k \) και \( N = \sum_k \text{support}_k \)
είναι το σύνολο των δειγμάτων, τότε για μια μετρική \( m_k \) ανά κλάση:

$$
m_{\text{weighted}} = \frac{1}{N} \sum_{k=1}^{K} m_k \cdot \text{support}_k
$$

- Κλάσεις με **πολλά δείγματα** επηρεάζουν περισσότερο το weighted avg.
- Κλάσεις με **λίγα δείγματα** επηρεάζουν λιγότερο.

Στο `classification_report`, το `weighted avg` δείχνει την «μέση» συμπεριφορά του μοντέλου
με βάση το **πόσα δείγματα** έχει κάθε κλάση (δηλαδή πιο κοντά στο τι συμβαίνει «ανά δείγμα»),
ενώ το `macro avg` είναι πιο κοντά σε μέση απόδοση «ανά κλάση».




In [ ]:
# Χωρίζουμε τα δεδομένα σε train / validation και εκπαιδεύουμε μοντέλο

# Εξάγουμε τα κείμενα (features) για το X
X = df["text"]

# Εξάγουμε τις ετικέτες και τις μετατρέπουμε σε 0/1
# (df["label"] == "spam") δημιουργεί True/False
# astype(int) μετατρέπει True→1, False→0
y = (df["label"] == "spam").astype(int)  # 1 = spam, 0 = ham

# Χωρίζουμε τα δεδομένα σε train (80%) και validation (20%)
# test_size=0.2 σημαίνει 20% για validation, 80% για train
# random_state=0 κάνει το χώρισμα αναπαράξιμο (ίδιο κάθε φορά που το τρέχουμε)
# stratify=y διασφαλίζει ότι και στα δύο σύνολα υπάρχει ίδια αναλογία ham/spam
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=0,
    stratify=y,
)

# Εκτυπώνουμε τα μεγέθη των δύο συνόλων
print("Μέγεθος train:", X_train.shape[0])
print("Μέγεθος validation:", X_val.shape[0])

# Δημιουργούμε ένα Pipeline που συνδυάζει δύο βήματα:
# 1. TfidfVectorizer: μετατρέπει κείμενο σε TF-IDF features (αριθμητικές τιμές)
#    - max_features=10000: κρατάμε τις 10000 πιο συχνές λέξεις
#    - stop_words='english': αφαιρούμε τις συχνές αγγλικές λέξεις (the, a, is, κλπ)
# 2. MultinomialNB: ο ταξινομητής Naive Bayes
#    - alpha=1.0: παράμετρος εξομάλυνσης Laplace
pipe = Pipeline(
    steps=[
        ("tfidf", TfidfVectorizer(max_features=10000, stop_words='english')),
        ("clf", MultinomialNB(alpha=1.0)),
    ]
)

# Εκπαιδεύουμε το pipeline στο training set
# pipe.fit() μαθαίνει το TF-IDF και το Naive Bayes από τα training δεδομένα
pipe.fit(X_train, y_train)

# Προβλέπουμε τα labels για το validation set
# Κάθε κείμενο στο X_val ταξινομείται ως 0 (ham) ή 1 (spam)
y_val_pred = pipe.predict(X_val)

print("\nΤο μοντέλο έχει εκπαιδευτεί. Στη συνέχεια θα δούμε τα αποτελέσματα.")


In [ ]:
# Αποτελέσματα ταξινόμησης: Precision, Recall, F1-Score

# Εκτυπώνουμε λεπτομερή αποτελέσματα ταξινόμησης
# classification_report δείχνει precision, recall, f1-score για κάθε κλάση
# target_names=["ham", "spam"] δίνει ονόματα στις κλάσεις
# digits=3 εμφανίζει 3 δεκαδικά ψηφία
print("=" * 60)
print("Classification Report (validation set)")
print("=" * 60)
print(
    classification_report(
        y_val,
        y_val_pred,
        target_names=["ham", "spam"],
        digits=3,
    )
)

print("\n")

# Confusion matrix για να δούμε αναλυτικά τις σωστές / λανθασμένες προβλέψεις

# Υπολογίζουμε τον confusion matrix
# cm[i, j] = πλήθος δειγμάτων που ανήκουν στην κλάση i αλλά προβλέφθηκαν ως κλάση j
# labels=[0, 1] σημαίνει: 0=ham (σειρά/στήλη 0), 1=spam (σειρά/στήλη 1)
cm = confusion_matrix(y_val, y_val_pred, labels=[0, 1])

# Δημιουργούμε αντικείμενο ConfusionMatrixDisplay για την οπτικοποίηση
# display_labels=["ham", "spam"] θα μπουν ως ετικέτες στο διάγραμμα
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["ham", "spam"])

# Δημιουργούμε ένα νέο σχήμα matplotlib με ένα subplot
# fig = το σχήμα (παράθυρο), ax = ο άξονας (χώρος σχεδίασης)
fig, ax = plt.subplots()

# Σχεδιάζουμε τον confusion matrix
# ax=ax προσδιορίζει που θα σχεδιαστεί
# values_format="d" σημαίνει ότι οι τιμές εμφανίζονται ως ακέραιοι (integers)
disp.plot(ax=ax, values_format="d")

# Προσθέτουμε τίτλο στο διάγραμμα
ax.set_title("Confusion Matrix (validation set)")

# Προσαρμόζουμε τα περιθώρια του σχήματος για καλύτερη εμφάνιση
plt.tight_layout()

# Εμφανίζουμε το διάγραμμα
plt.show()


In [ ]:
# Παρατηρούμε τις πιο «χαρακτηριστικές» λέξεις για κάθε κλάση

# Εξάγουμε τον TF-IDF vectorizer από το pipeline (μετατρέπει κείμενο σε αριθμούς)
tfidf = pipe.named_steps["tfidf"]

# Εξάγουμε τον Naive Bayes ταξινομητή από το pipeline
clf = pipe.named_steps["clf"]

# Λαμβάνουμε τα ονόματα όλων των χαρακτηριστικών (λέξεων)
# get_feature_names_out() επιστρέφει τις λέξεις που εμφανίζονται στα δεδομένα
feature_names = np.array(tfidf.get_feature_names_out())

# Λαμβάνουμε τις log-πιθανότητες κάθε λέξης για κάθε κλάση
# shape: (2, n_features) όπου [0]=ham, [1]=spam
# feature_log_prob_[0] = log P(λέξη | ham)
# feature_log_prob_[1] = log P(λέξη | spam)
log_probs = clf.feature_log_prob_

# Αριθμός top λέξεων που θέλουμε να δούμε
top_n = 20

# Βρίσκουμε τις indices των 20 λέξεων με τη μεγαλύτερη πιθανότητα στο spam
# argsort()[::-1] ταξινομεί από μικρό προς μεγάλο, [::-1] αντιστρέφει σε φθίνουσα σειρά
spam_top_idx = np.argsort(log_probs[1])[::-1][:top_n]

# Βρίσκουμε τις indices των 20 λέξεων με τη μεγαλύτερη πιθανότητα στο ham
ham_top_idx = np.argsort(log_probs[0])[::-1][:top_n]

# Εκτυπώνουμε τις λέξεις που χαρακτηρίζουν περισσότερο το spam
# Χρησιμοποιούμε feature_names[spam_top_idx] για να πάρουμε τα ονόματα των λέξεων
print("Top λέξεις για την κλάση 'spam':")
print(feature_names[spam_top_idx])

# Εκτυπώνουμε τις λέξεις που χαρακτηρίζουν περισσότερο το ham
print("\nTop λέξεις για την κλάση 'ham':")
print(feature_names[ham_top_idx])


In [ ]:
# Πειραματισμός με την παράμετρο εξομάλυνσης alpha
# Σημαντικό: Χρησιμοποιούμε την ίδια ρύθμιση TfidfVectorizer όπως και το αρχικό μοντέλο
# (με stop_words='english') για να έχουμε δίκαιη σύγκριση

alphas = [0.01, 0.1, 0.5, 1.0, 5.0, 10.0]
results = []

for alpha in alphas:
    pipe_a = Pipeline(
        steps=[
            ("tfidf", TfidfVectorizer(max_features=10000, stop_words='english')),
            ("clf", MultinomialNB(alpha=alpha)),
        ]
    )
    pipe_a.fit(X_train, y_train)
    y_val_pred_a = pipe_a.predict(X_val)

    acc = accuracy_score(y_val, y_val_pred_a)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_val,
        y_val_pred_a,
        average="binary",
        pos_label=1,
        zero_division=0,
    )

    results.append((alpha, acc, prec, rec, f1))

alpha_results = pd.DataFrame(
    results,
    columns=["alpha", "accuracy", "precision", "recall", "f1"],
)
alpha_results


## Εξομάλυνση Laplace (Alpha Parameter) στο Multinomial Naive Bayes

### Τι είναι το Alpha;

Στο **Multinomial Naive Bayes**, υπολογίζουμε τις πιθανότητες κάθε λέξης σε κάθε κλάση:

$$
P(x_i \mid y) = \frac{\text{count of word } i \text{ in class } y}{\text{total words in class } y}
$$

**Πρόβλημα**: Τι συμβαίνει αν μια λέξη **δεν εμφανίζεται ποτέ** σε ένα training set αλλά εμφανίζεται σε ένα νέο μήνυμα;
Τότε θα έχουμε $P(x_i \mid y) = 0$, και ολόκληρη η posterior πιθανότητα γίνεται μηδέν!
Αυτό είναι πολύ αυστηρό και μη ρεαλιστικό.

Η **εξομάλυνση Laplace** (Laplace smoothing) με παράμετρο $\alpha$ αντιμετωπίζει αυτό το πρόβλημα προσθέτοντας έναν μικρό αριθμό στους μετρητές:

$$
P(x_i \mid y) = \frac{\text{count of word } i \text{ in class } y + \alpha}{\text{total words in class } y + \alpha \cdot V}
$$

όπου $V$ είναι το μέγεθος του λεξιλογίου (πλήθος μοναδικών λέξεων).

**Ερμηνεία**:

- **$\alpha = 0$** (χωρίς εξομάλυνση): Αν μια λέξη δεν εμφανίζεται σε μια κλάση, η πιθανότητα της είναι 0.
  Αυτό έχει ως αποτέλεσμα φτωχή γενίκευση σε νέα δεδομένα.

- **$\alpha$ μεγάλο** (π.χ. $\alpha = 10$): Δίνουμε πολύ μεγάλο βάρος στην εξομάλυνση.
  Όλες οι λέξεις γίνονται περισσότερο ισοδύναμες, και το μοντέλο γίνεται πιο "συντηρητικό".
  Κάνει λιγότερο χρήση των πληροφοριών που πραγματικά υπάρχουν στα δεδομένα.

- **$\alpha$ μικρό** (π.χ. $\alpha = 0.01$): Δίνουμε μικρό βάρος στην εξομάλυνση.
  Το μοντέλο βασίζεται περισσότερο στις πραγματικές συχνότητες του training set.
  Αλλά κινδυνεύει να "overfittάρει" (να θυμάται τα δεδομένα αντί να μαθαίνει γενικούς κανόνες).

---

### Γιατί βλέπουμε αυτά τα αποτελέσματα;

Ας δούμε τα δεδομένα από το πείραμα (με `stop_words='english'`):

| alpha | accuracy | precision | recall | f1 |
|-------|----------|-----------|--------|-----|
| 0.01  | 0.9758   | 0.9357    | 0.8792 | 0.9066 |
| 0.10  | 0.9776   | 0.9559    | 0.8725 | 0.9123 |
| 0.50  | 0.9794   | 0.9922    | 0.8523 | 0.9170 |
| 1.00  | 0.9713   | 1.0000    | 0.7852 | 0.8797 |
| 5.00  | 0.8996   | 1.0000    | 0.2483 | 0.3978 |
| 10.00 | 0.8673   | 1.0000    | 0.0067 | 0.0133 |

**Παρατηρήσεις**:

#### 1. **Σε χαμηλό α (0.01 - 0.50): Ισορροπημένη απόδοση**

- Η **recall** είναι υψηλή (~0.85-0.88): Το μοντέλο βρίσκει τα περισσότερα spam μηνύματα.
- Το **F1-score** είναι υψηλό (~0.91): Καλή συνολική απόδοση.
- Αυτά τα αποτελέσματα είναι **ρεαλιστικά** και **πρακτικά** για χρήση.
- **Βέλτιστο**: $\alpha = 0.50$ με F1 = 0.9170 και χαμηλότερα false positives (Precision = 0.9922).

**Γιατί;** Ο μικρός $\alpha$ επιτρέπει στο μοντέλο να **εκμεταλλεύεται τις πραγματικές λέξεις spam** 
που εμφανίζονται συχνά στα δεδομένα (π.χ. "winner", "prize", "free"). Ταυτόχρονα, η εξομάλυνση 
εμποδίζει την εντελώς μηδενική πιθανότητα.

#### 2. **Σε μεσαίο α (1.00): Precision αυξάνεται, Recall μειώνεται**

- Precision = 1.000 (τέλειο): Όλα τα spam που προβλέψαμε ήταν σωστά.
- Recall πέφτει δραματικά: Στο 0.7852 (από 0.8792 στο α=0.01).
- F1-score = 0.8797: Χειρότερο από τα χαμηλά α.

**Γιατί;** Ο αυξημένος $\alpha$ «εξομαλύνει» τις πιθανότητες των λέξεων, 
κάνοντας τις διαφορές μεταξύ ham και spam λιγότερο έντονες. 
Το μοντέλο γίνεται **πιο συντηρητικό** στη διάγνωση spam, αποφεύγει false positives 
αλλά χάνει πολλά πραγματικά spam (false negatives).

#### 3. **Σε μεγάλο α (5.00 - 10.00): Κατάρρευση του μοντέλου**

- Recall → 0.2483 (για $\alpha = 5.0$): Το μοντέλο **δεν βρίσκει σχεδόν καν spam**.
- Recall → 0.0067 (για $\alpha = 10.0$): Το μοντέλο **σχεδόν ποτέ δεν προβλέπει spam**.
- F1-score πλησιάζει το 0: Ο ταξινομητής είναι άχρηστος.

**Γιατί;** Ο **εξαιρετικά μεγάλος** $\alpha$ κάνει όλες τις λέξεις ισοδύναμες μεταξύ ham και spam. 
Η εξομάλυνση γίνεται τόσο ισχυρή που **ακυρώνει** την πληροφορία που εμπεριέχεται στα πραγματικά δεδομένα. 
Το μοντέλο **δεν μπορεί να διακρίνει** τα δύο.

---

### Συμπέρασμα: Επιλογή του βέλτιστου α

Για αυτό το SMS Spam dataset με `stop_words='english'`:

- **Βέλτιστο**: $\alpha = 0.50$  
  F1-score = 0.9170 (υψηλότερο), Precision = 0.9922, Recall = 0.8523.
  Συμφωνία: κλάση spam ανιχνεύεται με λογική και περιορίζονται τα false alarms.

- **Πολύ καλό**: $\alpha \in [0.01, 0.10]$  
  F1-score ≈ 0.91, ελαφρώς υψηλότερη recall (~0.88), αλλά περισσότερα false positives.

- **Αποδεκτό αλλά όχι βέλτιστο**: $\alpha = 1.00$  
  F1-score = 0.8797, μεσαία απόδοση. Recall πέφτει σημαντικά.

- **Κακό**: $\alpha \geq 5.00$  
  Ο μοντέλος σπάει, recall → 0, F1-score → 0.

**Γενικά κανόνια**:

- Ξεκινάμε συχνά με **$\alpha = 1.0$** (η "απλή" εξομάλυνση Laplace).
- Για αυτό το dataset, η πειραματική εξερεύνηση δείχνει ότι **χαμηλότερα α** (0.01-0.50) δουλεύουν **σημαντικά καλύτερα**.
- Η εξέταση του trade-off μεταξύ Precision και Recall είναι **κρίσιμη**: 
  - Αν θέλουμε υψηλή Recall (να ανιχνεύσουμε όσο περισσότερο spam γίνεται), **χρησιμοποιούμε χαμηλό α** (π.χ. 0.01).
  - Αν θέλουμε υψηλή Precision (να αποφύγουμε false alarms), **χρησιμοποιούμε μεσαίο α** (π.χ. 0.50).


In [ ]:
# Γραφική απεικόνιση F1 σε συνάρτηση με το alpha

fig, ax = plt.subplots()
ax.plot(alpha_results["alpha"], alpha_results["f1"], marker="o")
ax.set_xscale("log")
ax.set_xlabel("alpha (log scale)")
ax.set_ylabel("F1 score (spam class)")
ax.set_title("Επίδραση της παραμέτρου alpha στο F1")
plt.tight_layout()
plt.show()

In [ ]:
# Δοκιμάζουμε το μοντέλο με βέλτιστο alpha σε μερικά νέα μηνύματα
# (Χρησιμοποιούμε α=0.01 που έδωσε τα καλύτερα αποτελέσματα: F1=0.925)

# Εκπαιδεύουμε ένα νέο μοντέλο με α=0.01
pipe_best = Pipeline(
    steps=[
        ("tfidf", TfidfVectorizer(max_features=10000, stop_words='english')),
        ("clf", MultinomialNB(alpha=0.5)),
    ]
)
pipe_best.fit(X_train, y_train)

example_messages = [
    "WINNER!! You have won a 1000$ cash prize. Call now to claim.",
    "Hey, are we still on for coffee tomorrow?",
    "FREE entry in 2 a weekly competition to win FA Cup final tickets. Text WIN to 87121 now!",
    "I'll be there in 10 minutes.",
]

X_new = pd.Series(example_messages)
y_pred_new = pipe_best.predict(X_new)
y_proba_new = pipe_best.predict_proba(X_new)

for text, label, proba in zip(example_messages, y_pred_new, y_proba_new):
    cls = "spam" if label == 1 else "ham"
    p_ham, p_spam = proba[0], proba[1]

    print("-" * 72)
    print("Μήνυμα:")
    print(text)
    print()
    print(f"→ Πρόβλεψη: {cls.upper()}")
    print(f"   P(ham | x)  = {p_ham:.3f}")
    print(f"   P(spam | x) = {p_spam:.3f}")
